In [136]:
import numpy as np
from numpy import linalg as LA
from numpy import matlib
from numpy.linalg import matrix_power
from timeit import default_timer as timer

In [137]:
D = 1000
U = np.zeros((D, 9))
V = np.zeros((D, 9))

In [138]:
for i in range(0, 9):
    U[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);
    V[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);

In [139]:
A = np.matmul(U,V.transpose())
eigenvalues, eigenvectors = LA.eig(A)
max_eigenvalue = max(np.absolute(eigenvalues))
A = np.dot(0.8, np.divide(A, max_eigenvalue))

In [140]:
mu = np.random.rand(D,1)/D
w = 1

In [141]:
import math
def comp_lambda(cur_t, cur_event, last_t, lambdat,w,mu,A):
    lambda_comp = mu + (lambdat - mu)*(math.exp(-w * (cur_t - last_t)))
    if (cur_event):
        lambda_comp = lambda_comp + np.expand_dims(A[cur_event, :].T,axis=1)
    return lambda_comp

In [142]:
import random 

def generate_samples(w, mu, A, num_sequences, max_events_per_sequence):
    start_time = timer()
    hp_samples = []

    for i in range(1, num_sequences+1):
        t = 0
        timestamp_and_event = []
        lambdat = mu
        lambdat_sum = np.sum(lambdat)
        
        while len(timestamp_and_event) < max_events_per_sequence:
            rand_u = random.uniform(0, 1)
            dt = np.random.exponential(1/lambdat_sum)            
            lambda_ts = comp_lambda(t+dt, [], t, lambdat,w,mu,A);
            lambdats_sum = np.sum(lambda_ts);
                        
            if (rand_u > (lambdats_sum/lambdat_sum)):
                t = t+dt
                lambdat = lambda_ts

            else:
                u = random.uniform(0, 1) * lambdats_sum
                lambda_sum = 0
                
                d = 0
                for d in range(1, len(mu)):
                    lambda_sum = lambda_sum + lambda_ts[d]
                    if(lambda_sum >= u):
                        break
            
                lambdat = comp_lambda(t+dt, d, t, lambdat, w, mu, A)
                t = t+dt
                timestamp_and_event.append([t,d])

            lambdat_sum = np.sum(lambdat)
        
        hp_samples.append(timestamp_and_event[0:])
        
        if (i%10 == 0):
            print("samples = " + str(i)+ "/" + str(num_sequences) + ", time = " 
                  + "{:.2f}".format(timer() - start_time) + " sec.")
    
    return hp_samples

In [143]:
num_sequences = 250
max_events_per_sequence = 100
hawkes_process_samples = generate_samples(w, mu, A, num_sequences, max_events_per_sequence)

samples = 10/250, time = 1.16 sec.
samples = 20/250, time = 2.31 sec.
samples = 30/250, time = 3.57 sec.
samples = 40/250, time = 4.78 sec.
samples = 50/250, time = 6.01 sec.
samples = 60/250, time = 7.24 sec.
samples = 70/250, time = 8.50 sec.
samples = 80/250, time = 9.78 sec.
samples = 90/250, time = 11.08 sec.
samples = 100/250, time = 12.41 sec.
samples = 110/250, time = 13.65 sec.
samples = 120/250, time = 15.09 sec.
samples = 130/250, time = 16.51 sec.
samples = 140/250, time = 17.88 sec.
samples = 150/250, time = 19.32 sec.
samples = 160/250, time = 20.64 sec.
samples = 170/250, time = 22.09 sec.
samples = 180/250, time = 23.29 sec.
samples = 190/250, time = 24.57 sec.
samples = 200/250, time = 25.88 sec.
samples = 210/250, time = 27.11 sec.
samples = 220/250, time = 28.43 sec.
samples = 230/250, time = 29.89 sec.
samples = 240/250, time = 31.21 sec.
samples = 250/250, time = 32.31 sec.


In [144]:
'''
timestamp = 1x100 double
event = 1x100 event            
'''
print (len(hawkes_process_samples))
print (len(hawkes_process_samples[0]))
print (hawkes_process_samples[0])

250
100
[[0.2774139477706948, 679], [1.320759198959638, 63], [1.3321611117267054, 614], [3.021332829628583, 689], [3.1589892836052105, 124], [7.351345813456013, 746], [8.480493041541097, 742], [8.803321622277723, 792], [9.373998763964295, 794], [9.386754273797214, 889], [9.410002617354484, 823], [9.487184734431798, 838], [9.496336088959353, 772], [9.524383962939327, 838], [9.906103194605462, 998], [9.997187158562765, 834], [10.025184958823083, 867], [10.4675580801814, 893], [10.56270890838354, 919], [10.794444186543572, 362], [11.111819177182008, 799], [11.255526306357002, 252], [11.347184362284304, 271], [11.375825608249354, 693], [11.635647478544056, 209], [11.637976973373904, 321], [12.13778176394819, 235], [12.194799623055424, 200], [12.46386361375082, 807], [12.866668132099244, 264], [13.672386256312024, 233], [14.506144607143732, 464], [14.916553888945845, 736], [14.939615682650217, 666], [14.964419715138197, 546], [15.102144114646327, 608], [15.723522687900932, 645], [17.7171797

In [145]:
def kernel_g(dt, w):
    g = np.multiply(w, np.exp(np.multiply(-w, dt)))
    g[g > 1] = 0
    return g

In [146]:
def kernel_int_g(dt, w):
    G = np.subtract(1, np.exp(np.multiply(-w, dt)))
    G[G<0] = 0
    return G

In [167]:
def real_err(A, A_m):
    err_1 = np.divide(abs(A - A_m), A)
    err_2 = abs(A - A_m) 
    
    err_1 = np.nan_to_num(err_1)
    err_1 = convert_inf(err_1, np.isinf(err_1))
    
    err = np.multiply(err_1, ((A!=0)*1).astype(float)) + np.multiply(err_2, ((A==0)*1).astype(float))
    err = np.sum(err.flatten())/float(A.size)
    
    return err

In [160]:
def convert_inf(A, inf_A):
    for i in range(0, len(inf_A)):
        for j in range(0, len(inf_A[0])):
            if(inf_A[i][j]):
                A[i][j] = 0
    return A

In [188]:
def optimize_mu(hp_samples, D, w, rho, num_iter_1, num_iter_2, thold, real_A):
    A_m = np.random.rand(D,D);
    eigenvalues_m, eigenvectors_m = LA.eig(A_m)
    max_eigenvalue_m = max(np.absolute(eigenvalues_m))
    A_m = np.dot(0.8, np.divide(A_m, max_eigenvalue_m))
    Iteration_Err = np.zeros((num_iter_2 + 2, num_iter_1 + 2))
    
    
    # reshape may required in mu_m
    mu_m = np.divide(np.random.rand(D, 1), D)
    UL = np.zeros((D, D))
    ZL = np.zeros((D, D))
    US = np.zeros((D, D))
    ZS = np.zeros((D, D))
    
    for i in range(0, num_iter_1 + 1):
        rho = rho * 1.1
        
        for j in range(0, num_iter_2 + 1):
            print ("No. " + str(i + 1) + " outter while iteration | No. " 
                   + str(j + 1) +  " inner while iteration")
            A_m, mu_m, RelErr = update_mu(A_m, mu_m, hp_samples, UL, ZL, US, ZS, w, rho, D, real_A)
            Iteration_Err[j + 1, i + 1] = RelErr
        
        s, v, d = np.linalg.svd(np.add(A_m, US))
        v = np.subtract(v, thold / rho)
        v[v < 0] = 0
        ZL = s * v * d.T
        UL = UL + np.subtract(A_m, ZL) #changed A_m-ZL to np.subtract
        
        #idk looks good to me lol
        tmp = np.subtract(abs(np.add(A_m, US)), thold / rho) # may have error 
        
        tmp[tmp < 0] = 0
        ZS = (np.multiply(np.sign(np.add(A_m, US)), tmp))
        US = np.add(US, np.subtract(A_m, ZS))
        
    return A_m, mu_m, Iteration_Err

In [189]:
def update_mu (A_m, mu_m, hp_samples, UL, ZL, US, ZS, w, rho, D, real_A):
    num_samples = len(hp_samples)
    mu_numerator = np.zeros((D, 1))
    
    C = np.zeros((len(A_m), len(A_m[0])))
    A_Step = np.add(np.zeros((len(A_m), len(A_m[0]))), 2 * rho)
    B = np.add(np.add(np.zeros((len(A_m), len(A_m[0]))), np.multiply(rho, np.subtract(UL, ZL))), 
               np.multiply(rho, np.subtract(US, ZS)))
    
    for s in range(0, num_samples):
        cur_hp_samples = hp_samples[s]
        timestamp = [i[0] for i in cur_hp_samples]
        event = [i[1] for i in cur_hp_samples]
        tc = timestamp[len(timestamp) - 1]
        nc = len(event)
        dt = np.subtract(tc, timestamp)
        
        for i in range(0, nc):
            ui = event[i]
            ti = timestamp[i]
            int_g = kernel_int_g(dt, w)
            
            # Todo: modify matrix B (Incomplete)
            # B(ui,:) = B(ui,:) + double(A_m(ui,:)>0).*repmat(int_g(i),[1,D]);            
            B[ui,:] = B[ui,:] + np.multiply((A_m[ui,:]>0).astype(float), np.matlib.repmat(int_g[i], 1, D))
            
            pii = []
            pij = []
            ag_arr = []
            
            if (i > 0):
                tj = timestamp[0 : i]
                uj = event[0 : i]
                kn_g = kernel_g(np.subtract(ti, tj), w)
                ag_arr = np.multiply(A_m[uj, ui], kn_g.T)
                
            pii = np.divide(mu_m[ui], mu_m[ui] + sum(ag_arr))
            
            if(i > 0):
                pij = np.divide(ag_arr, mu_m[ui] + sum(ag_arr))
                if (len(pij) != 0 and sum(pij) > 0):
                    for j in range(0, len(uj)):
                        uuj = uj[j]                        
                        C[uuj, ui] = C[uuj, ui] - pij[j] ## (Incomplete) Question: what we have at the end ??? value or vector ??
            
            mu_numerator[ui] = np.add(mu_numerator[ui], pii)
            
    mu = np.divide(mu_numerator, np.add(np.zeros((D, 1)), tc))
    sqrt_eq = np.sqrt(np.subtract(matrix_power(B, 2), np.multiply(4, np.multiply(A_Step, C))))
    A  = np.divide(np.add(np.multiply(-1, B), sqrt_eq), np.multiply(2, A_Step))
    RelErr = real_err(real_A, A)
    
    print ("non-zero in mu = " + str(np.count_nonzero(mu)))
    print ("non-zero in C = "  + str(np.count_nonzero(C)))
    print ("non-zero in B = "  + str(np.count_nonzero(B)) + ", non-zero in sqrt = " + str(np.count_nonzero(sqrt_eq)))
    print ("real error = " + "{:.4f}".format(RelErr) 
#            + ", correlation = " + "{:.4f}".format()  # (Incomplete)
           + "#non-zero in A = " + str(np.count_nonzero(A)))
            
    A = np.nan_to_num(A)
    A = convert_inf(A, np.isinf(A))
    
    return A, mu, RelErr

In [190]:
#start optimization
num_iter_1 = 2;    # Number of Iteration of the First while loop (while k = 1,2 ...)
num_iter_2 = 7;    # Number of Iteration of the Second while loop (while not converge)

rho = 0.09;
thold = 1;         # thershold value

In [191]:
[x,y,Iteration_Err] = optimize_mu(hawkes_process_samples,D,w,rho,num_iter_1,num_iter_2,thold,A);

No. 1 outter while iteration | No. 1 inner while iteration


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in divide
  
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in divide
  


non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999000, non-zero in sqrt = 999000
real error = 2224459.1543#non-zero in A = 999000
No. 1 outter while iteration | No. 2 inner while iteration
non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999000, non-zero in sqrt = 999000
real error = 2224459.1602#non-zero in A = 999000
No. 1 outter while iteration | No. 3 inner while iteration
non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999000, non-zero in sqrt = 999000
real error = 2224459.1603#non-zero in A = 999000
No. 1 outter while iteration | No. 4 inner while iteration
non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999000, non-zero in sqrt = 999000
real error = 2224459.1603#non-zero in A = 999000
No. 1 outter while iteration | No. 5 inner while iteration
non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999000, non-zero in sqrt = 999000
real error = 2224459.1603#non-zero in A = 999000
No. 1 outter while iteration | No. 6 inner while 

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in sqrt


non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999001, non-zero in sqrt = 999001
real error = nan#non-zero in A = 999000
No. 2 outter while iteration | No. 2 inner while iteration
non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999001, non-zero in sqrt = 999001
real error = nan#non-zero in A = 999000
No. 2 outter while iteration | No. 3 inner while iteration
non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999001, non-zero in sqrt = 999001
real error = nan#non-zero in A = 999000
No. 2 outter while iteration | No. 4 inner while iteration
non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999001, non-zero in sqrt = 999001
real error = nan#non-zero in A = 999000
No. 2 outter while iteration | No. 5 inner while iteration
non-zero in mu = 999
non-zero in C = 605265
non-zero in B = 999001, non-zero in sqrt = 999001
real error = nan#non-zero in A = 999000
No. 2 outter while iteration | No. 6 inner while iteration
non-zero in mu = 999
non-zero in C 